# AMES housing prices prediction

### Initialization

In [1]:
import sys
sys.path.insert(0, '../..')

In [2]:
import numpy as np
from cerebros.simplecerebrosrandomsearch.simple_cerebros_random_search\
    import SimpleCerebrosRandomSearch
import pendulum
import pandas as pd
import tensorflow as tf
from cerebros.units.units import DenseUnit
from cerebros.denseautomlstructuralcomponent.dense_automl_structural_component\
    import zero_7_exp_decay, zero_95_exp_decay, simple_sigmoid
from ast import literal_eval

/Users/sasha/jax-metal/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Some architectural and training hyperparameters

In [2]:
activation = "gelu"
predecessor_level_connection_affinity_factor_first = 19.613
predecessor_level_connection_affinity_factor_main = 0.5518
max_consecutive_lateral_connections = 34
p_lateral_connection = 0.36014
num_lateral_connection_tries_per_unit = 11
learning_rate = 0.095
epochs = 145
batch_size = 634
maximum_levels = 5
maximum_units_per_level = 5
maximum_neurons_per_unit = 25

### Importing raw data, preparing the training set

In [3]:
# Replace the data file with your data 
# and tell us what column you want to predict
data_file = 'ames.csv'
prediction_target_column = 'price'

In [4]:
# Load raw data, grab numerical columns from it
raw_data = pd.read_csv(data_file)
needed_cols = [col for col in raw_data.columns if raw_data[col].dtype != 'object']
data_numeric = raw_data[needed_cols].fillna(0).astype(float)

# Pop the target column into labels
label = raw_data.pop(prediction_target_column)

# Convert the rest into features
data_np = data_numeric.values

# Logging
TIME = pendulum.now(tz='America/New_York').__str__()[:16]\
    .replace('T', '_')\
    .replace(':', '_')\
    .replace('-', '_')
PROJECT_NAME = f'{TIME}_cerebros_auto_ml_ames_test'

# Labelled train dataset
tensor_x = tf.constant(data_np)
training_x = [tensor_x]
train_labels = [label.values]

# Input and output shapes
INPUT_SHAPES = [training_x[i].shape[1] for i in np.arange(len(training_x))]
OUTPUT_SHAPES = [1]  

### Running Cerebros AutoML session

Instantiate Cerebros search

In [5]:
meta_trial_number = 42 # can be ignored unless in distributed training
#
cerebros_automl = SimpleCerebrosRandomSearch(
    unit_type=DenseUnit,
    input_shapes=INPUT_SHAPES,
    output_shapes=OUTPUT_SHAPES,
    training_data=training_x,
    labels=train_labels,
    validation_split=0.35,
    direction='minimize',
    metric_to_rank_by='val_root_mean_squared_error',
    minimum_levels=2,
    maximum_levels=maximum_levels,
    minimum_units_per_level=1,
    maximum_units_per_level=maximum_units_per_level,
    minimum_neurons_per_unit=1,
    maximum_neurons_per_unit=maximum_neurons_per_unit,
    activation=activation,
    final_activation=None,
    number_of_architecture_moities_to_try=7,
    number_of_tries_per_architecture_moity=1,
    minimum_skip_connection_depth=1,
    maximum_skip_connection_depth=7,
    predecessor_level_connection_affinity_factor_first=predecessor_level_connection_affinity_factor_first,
    predecessor_level_connection_affinity_factor_first_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_main=predecessor_level_connection_affinity_factor_main,
    predecessor_level_connection_affinity_factor_main_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_decay_main=zero_7_exp_decay,
    seed=8675309,
    max_consecutive_lateral_connections=max_consecutive_lateral_connections,
    gate_after_n_lateral_connections=3,
    gate_activation_function=simple_sigmoid,
    p_lateral_connection=p_lateral_connection,
    p_lateral_connection_decay=zero_95_exp_decay,
    num_lateral_connection_tries_per_unit=num_lateral_connection_tries_per_unit,
    learning_rate=learning_rate,
    loss='mse',
    metrics=[tf.keras.metrics.RootMeanSquaredError()],
    epochs=epochs,
    project_name=f"{PROJECT_NAME}_meta_{meta_trial_number}",
    # use_multiprocessing_for_multiple_neural_networks=False,  # pull this param
    model_graphs='model_graphs',
    batch_size=batch_size,
    meta_trial_number=meta_trial_number)
#

Best model search

In [6]:
result = cerebros_automl.run_random_search()
best_model_found = cerebros_automl.get_best_model()

SimpleCerebrosRandomSearch.input_shapes: [39]
nan
>nnf>ceil
k is: 0 value is: [{'1': <class 'cerebros.units.units.InputUnit'>}]
0
k is: 1 value is: [{'7': <class 'cerebros.units.units.DenseUnit'>}]
1
Trying to create level 1
We think level 1's predecessors are: [0]
k is: 2 value is: [{'9': <class 'cerebros.units.units.DenseUnit'>}, {'6': <class 'cerebros.units.units.DenseUnit'>}, {'6': <class 'cerebros.units.units.DenseUnit'>}, {'7': <class 'cerebros.units.units.DenseUnit'>}, {'20': <class 'cerebros.units.units.DenseUnit'>}]
2
Trying to create level 2
We think level 2's predecessors are: [0, 1]
k is: 3 value is: [{'1': <class 'cerebros.units.units.FinalDenseUnit'>}]
3
Trying to create Final level 3
Trying to create level 3
We think level final level 3's predecessors are: [0, 1, 2]
levels:
[0, 1, 2, 3]
{'0': 'InputUnitModule'}
InputLevel.input_shapes [39]
{'7': <class 'cerebros.units.units.DenseUnit'>}
{'9': <class 'cerebros.units.units.DenseUnit'>}
{'6': <class 'cerebros.units.units.De

Debug: successor_connectivity_errors_2d []
materialize:_NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000003_tr_0_FinalDenseUnit_0000000000000003_tr_0_0 called
materialized network layers
[<KerasTensor: shape=(None, 7) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000002_tr_0_DenseUnit_0000000000000002_tr_0_3_dns_')>, <KerasTensor: shape=(None, 6) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000002_tr_0_DenseUnit_0000000000000002_tr_0_2_dns_')>, <KerasTensor: shape=(None, 6) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000002_tr_0_DenseUnit_0000000000000002_tr_0_2_dns_')>, <KerasTensor: shape=(None, 6) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000002_tr_0_DenseUnit_0000000000000002_tr_0_1_dns_')>, <KerasTensor: shape=(None, 20) dtype=float32 (created by layer 'N

INFO:tensorflow:Assets written to: 2023_10_18_05_07_cerebros_auto_ml_test_meta_42/models/tr_0000000000000000_subtrial_0000000000000000/assets


returning trial 0 oracles
             loss  root_mean_squared_error      val_loss  \
0    4.089416e+10            202223.031250  5.165654e+10   
1    4.087155e+10            202167.140625  2.209968e+10   
2    4.082158e+10            202043.531250  5.957841e+09   
3    4.074038e+10            201842.468750  1.775967e+10   
4    4.062242e+10            201550.046875  2.814659e+10   
..            ...                      ...           ...   
140  8.479907e+07              9208.641602  1.029329e+08   
141  1.117698e+08             10572.125977  1.098720e+08   
142  1.076630e+08             10376.078125  1.041488e+08   
143  1.863405e+08             13650.659180  6.269663e+07   
144  1.365292e+08             11684.570312  3.549466e+08   

     val_root_mean_squared_error  trial_number  subtrial_number  \
0                  227280.765625             0                0   
1                  148659.609375             0                0   
2                   77187.054688             0      

Debug: successor_connectivity_errors_2d []
materialize:_NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000005_tr_0_FinalDenseUnit_0000000000000005_tr_0_0 called
materialized network layers
[<KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000004_tr_0_DenseUnit_0000000000000004_tr_0_0_dns_')>, <KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000004_tr_0_DenseUnit_0000000000000004_tr_0_0_dns_')>, <KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000004_tr_0_DenseUnit_0000000000000004_tr_0_0_dns_')>, <KerasTensor: shape=(None, 25) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000004_tr_0_DenseUnit_0000000000000004_tr_0_1_dns_')>, <KerasTensor: shape=(None, 25) dtype=float32 (created by layer '

...

INFO:tensorflow:Assets written to: 2023_10_18_05_07_cerebros_auto_ml_test_meta_42/models/tr_0000000000000000_subtrial_0000000000000000/assets


returning trial 0 oracles
             loss  root_mean_squared_error      val_loss  \
0    4.089436e+10            163152.062500  4.680450e+12   
1    4.086470e+10            202150.203125  5.171111e+12   
2    4.079606e+10            201980.343750  9.474936e+11   
3    4.067572e+10            201682.218750  1.214808e+11   
4    4.049111e+10            201224.031250  5.183052e+09   
..            ...                      ...           ...   
140  2.090977e+08             14460.212891  1.727072e+07   
141  1.919764e+08             13855.555664  3.152457e+07   
142  1.347310e+08             11607.368164  4.496242e+07   
143  1.164655e+08             10791.917969  5.396657e+07   
144  9.947259e+07              9973.594727  4.900759e+07   

     val_root_mean_squared_error  trial_number  subtrial_number  \
0                   2.163435e+06             0                0   
1                   2.274008e+06             0                0   
2                   9.733928e+05             0      

inputs
KerasTensor(type_spec=TensorSpec(shape=(None, 39), dtype=tf.float32, name='NeuralNetworkFuture_0000000000000nan_tr_0_InputLevel_0000000000000000_tr_0_InputUnit_0000000000000000_tr_0_0_inp'), name='NeuralNetworkFuture_0000000000000nan_tr_0_InputLevel_0000000000000000_tr_0_InputUnit_0000000000000000_tr_0_0_inp', description="created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_InputLevel_0000000000000000_tr_0_InputUnit_0000000000000000_tr_0_0_inp'")

outputs
KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), name='NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000004_tr_0_FinalDenseUnit_0000000000000004_tr_0_0_dns_/BiasAdd:0', description="created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000004_tr_0_FinalDenseUnit_0000000000000004_tr_0_0_dns_'")
Model: "NeuralNetworkFuture_0000000000000nan_tr_0_nn_materialized"
______________________________________________________________________________

INFO:tensorflow:Assets written to: 2023_10_18_05_07_cerebros_auto_ml_test_meta_42/models/tr_0000000000000000_subtrial_0000000000000000/assets


returning trial 0 oracles
             loss  root_mean_squared_error      val_loss  \
0    4.089482e+10            163069.968750  4.650873e+12   
1    4.087143e+10            202166.843750  1.184588e+10   
2    4.081549e+10            202028.453125  4.440071e+09   
3    4.071594e+10            201781.906250  1.336033e+10   
4    4.056640e+10            201411.015625  1.383568e+11   
..            ...                      ...           ...   
140  1.901007e+08             13787.702148  3.854720e+08   
141  2.233356e+08             14944.417969  3.321973e+08   
142  2.257313e+08             15024.356445  2.043789e+08   
143  1.731107e+08             13157.153320  1.383580e+08   
144  1.558518e+08             12484.060547  1.272702e+08   

     val_root_mean_squared_error  trial_number  subtrial_number  \
0                   2.156588e+06             0                0   
1                   1.088388e+05             0                0   
2                   6.663386e+04             0      

In [7]:
trainable_params = np.sum([np.prod(w.get_shape()) for w in best_model_found.trainable_weights])
non_trainable_params = np.sum([np.prod(w.get_shape()) for w in best_model_found.non_trainable_weights])
total_params = trainable_params + non_trainable_params
#
print(f"Best model found: {total_params} total parameters ({trainable_params} trainable, {non_trainable_params} non-trainable)")
#
print(f"Best accuracy is ({cerebros_automl.metric_to_rank_by}): {result}")

Best model found: 100376 total parameters (84776 trainable, 15600 non-trainable)
Best accuracy is (val_root_mean_squared_error): 1080.707763671875
